In [2]:
import os
import pandas as pd
import faiss
import pickle
from sentence_transformers import SentenceTransformer
import openai
import numpy as np


In [3]:
import os
from dotenv import load_dotenv
import openai

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [4]:
df = pd.read_excel("행정안전부_민원안내정보_20250327.xlsx")


df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6022 entries, 0 to 6021
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   분류번호    6022 non-null   object
 1   사무명     6022 non-null   object
 2   사무개요    6022 non-null   object
 3   신청유형    6022 non-null   object
 4   신청자 자격  6020 non-null   object
 5   수수료     6021 non-null   object
 6   수수료내역   6018 non-null   object
 7   소관부처    6022 non-null   object
 8   담당부서    6022 non-null   object
 9   접수방법    6022 non-null   object
 10  접수/처리   6013 non-null   object
 11  근거법령    5981 non-null   object
 12  신청서     5124 non-null   object
 13  발급물 내용  918 non-null    object
 14  구비서류    5372 non-null   object
dtypes: object(15)
memory usage: 705.8+ KB


In [5]:
df.head(20)

,분류번호,사무명,사무개요,신청유형,신청자 자격,수수료,수수료내역,소관부처,담당부서,접수방법,접수/처리,근거법령,신청서,발급물 내용,구비서류
0,SG4CADM-2015,토지임야대장등본교부 다량신청,토지임야대장등본교부 민원에 대하여 여러 번지를 한번에 신청할 수 있는 민원입니다.,교부,누구나신청가능,NaN,NaN,국토교통부,공간정보제도과,인터넷,NaN,NaN,NaN,NaN,NaN
1,9740000-0009,공탁신청(공탁물지급청구 포함),"공탁 신청, 공탁금의 지급을 위한 민원입니다. 공탁은 공탁자가 법령에 규정된 원인에...",승인,누구나신청가능,수수료 없음,수수료 없음,대법원,사법등기국,"방문, 인터넷",1. 유형없음(총 0일)\n*접수:지방법원.지원(0일)\n*경유:\n*협의:\n*처...,"1. 공탁법(제4조, 제9조) \n2. 공탁규칙(제20조, 제21조, 제23조, 제...",금전공탁서(변제 등) 공탁사무 문서양식에 관한 예규 : 별지서식 1호의 1\n,NaN,"1. 유형없음\n< 민원인 제출서류 >\n(1) 인감증명서 신분증 가압류,압류결정문..."
2,9740000-0008,친양자입양취소신고,가정법원의 허가를 얻어 친양자 입양을 취소하기 위해 신청하는 민원입니다.,신고,누구나신청가능,수수료 없음,수수료 없음,대법원,가족관계등록과,"방문, 우편",1. 유형없음( 즉시)\n*접수:시.구.읍.면(0일)\n*경유:\n*협의:\n*처리...,"1. 가족관계의 등록 등에 관한 법률(제69조, 제70조)","제9호, 친양자 입양취소신고서\n",NaN,1. 유형없음\n< 민원인 제출서류 >\n(1) 친양자입양취소당사자의 가족관계등록부...
3,9740000-0007,친양자파양신고,가정법원의 허가를 얻어 친양자입양을 파양하고자 하는 사람이 재판의 확정일부터 1개월...,신고,누구나신청가능,수수료 없음,수수료 없음,대법원,가족관계등록과,"방문, 우편",1. 유형없음( 지체없이)\n*접수:시.구.읍.면(0일)\n*경유:\n*협의:\n*...,1. 가족관계의 등록 등에 관한 법률(제69조),"제7호, 친양자파양신고서\n",NaN,1. 유형없음\n< 민원인 제출서류 >\n(1) 친양자파양당사자의 가족관계등록부의 ...
4,9740000-0006,성·본계속사용신고,부가 혼인외 자를 인지할 때 인지 전의 자녀의 성과 본을 유지할 경우 그 취지를 가...,신고,누구나신청가능,수수료 없음,수수료 없음,대법원,가족관계등록과,"방문, 우편",1. 유형없음(총 0일)\n*접수:시.구.읍.면(0일)\n*경유:\n*협의:\n*처...,1. 가족관계의 등록 등에 관한 법률(제55조 제1항),"제3호, 성·본계속사용신고서\n",NaN,1. 유형없음\n< 민원인 제출서류 >\n(1) 성·본계속사용자의 가족관계등록부의 ...
5,9740000-0005,성·본변경신고,자의 복리를 위하여 가정법원의 허가를 얻어 자녀의 성.본을 변경하고자 하는 자가 재...,신고,누구나신청가능,수수료 없음,수수료 없음,대법원,가족관계등록과,"방문, 우편",1. 유형없음( 지체없이)\n*접수:시.구.읍.면(0일)\n*경유:\n*협의:\n*...,1. 가족관계의 등록 등에 관한 법률(제100조 제1항),"제34호, 성·본변경신고서\n",NaN,1. 유형없음\n< 민원인 제출서류 >\n(1) 성본변경신고서\n(2) 가정법원의 ...
6,9740000-0004,가족관계등록부등의 증명서 발급,"국민 개개인의 출생, 입양, 혼인, 사망 등 가족관계의 발생 및 변동에 관한 증명서...",증명,본인또는대리인신청,사례별 수수료,"증명서 1통당 1,000원 / 인터넷으로 발급받을 때는 무료임",대법원,가족관계등록과,"방문, 인터넷, 무인발급기","1. 유형없음( 즉시)\n*접수:동, 시.구.읍.면(0일)\n*경유:\n*협의:\n...",1. 가족관계의 등록 등에 관한 법률(제14조),"별지 제11호 서식, 가족관계 등록사항별 증명서 교부 등 신청서\n",NaN,1. 유형없음\n< 민원인 제출서류 >\n(1) 신분증 지참
7,9740000-0003,건물등기사항증명서 발급,건물에 관한 등기기록 사항의 전부 또는 일부를 증명하는 서면을 발급받기 위한 민원입...,증명,누구나신청가능,사례별 수수료,"방문신청 통당 1,200원/무인발급기 및 인터넷발급신청 통당 1,000원",대법원,부동산등기과,"방문, 인터넷, 무인발급기",1. 유형없음(총 0일)\n*접수:지방등기소(0일)\n*경유:\n*협의:\n*처리:...,NaN,NaN,NaN,NaN
8,9740000-0002,토지등기사항증명서 발급,토지에 관한 등기기록 사항의 전부 또는 일부를 증명하는 서면을 발급받기 위한 민원입...,증명,누구나신청가능,사례별 수수료,"방문신청 통당 1,200원/무인발급기 및 인터넷발급신청 통당 1,000원",대법원,부동산등기과,"방문, 인터넷, 무인발급기",1. 유형없음(총 0일)\n*접수:지방등기소(0일)\n*경유:\n*협의:\n*처리:...,NaN,NaN,NaN,NaN
9,9740000-0001,법인등기사항증명서 발급,"법인등기사항증명서를 발급받기 위한 민원입니다. 법인등기사항증명서는 법인(상사법인, ...",교부,누구나신청가능,사례별 수수료,"방문신청 통당 1,200원/무인발급기 및 인터넷발급신청 통당 1,000원",대법원,사법등기국,"방문, 인터넷",1. 유형없음(총 0일)\n*접수:지방등기소(0일)\n*경유:\n*협의:\n*처리:...,1. 상업등기법(제15조) \n2. 상업등기규칙(제26조),NaN,NaN,NaN


In [6]:
#담당부서 unique 몇개인지 찾는식
unique_departments = df['담당부서'].unique()
unique_department1s = df['소관부처'].unique()
print(f"Unique departments: {len(unique_departments)}")
print(f"Unique departments: {len(unique_department1s)}")

Unique departments: 753
Unique departments: 46


In [5]:
# 청크 생성
documents = []
metadatas = []
for idx, row in df.iterrows():
    chunk = "\n".join([f"[{col}] {row[col]}" for col in df.columns])
    documents.append(chunk)
    metadatas.append({"사무명": row["사무명"], "분류번호": row["분류번호"]})


In [6]:
# # 임베딩 생성
model = SentenceTransformer("jhgan/ko-sroberta-multitask")

# roberta 모델을 사용하여 문서 임베딩 생성 -> 출력 차원: 768

In [7]:
embeddings = model.encode(documents, show_progress_bar=True)
# # FAISS 저장
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# # 인덱스와 메타데이터 저장
faiss.write_index(index, "rag_index.faiss")
with open("rag_metadata.pkl", "wb") as f:
    pickle.dump((documents, metadatas), f)

Batches:   0%|          | 0/189 [00:00<?, ?it/s]

In [8]:
# 인덱스 및 메타데이터 로드
index = faiss.read_index("rag_index.faiss")
with open("rag_metadata.pkl", "rb") as f:
    documents, metadatas = pickle.load(f)

In [9]:
from openai import OpenAI
client = OpenAI(api_key = api_key)

In [10]:
# 검색 함수
def search_similar_documents(query, top_k=3):
    query_embedding = model.encode([query])
    D, I = index.search(query_embedding, top_k)
    return [(documents[i], metadatas[i]) for i in I[0]]

# 프롬프트 생성
def build_prompt(query, retrieved_docs):
    context = "\n\n".join([doc for doc, meta in retrieved_docs])
    return f"""다음은 민원 관련 문서입니다:\n\n{context}\n\n질문: {query}\n\n답변:"""

# 사용자 입력 → GPT 응답 생성
def answer_query(query):
    retrieved_docs = search_similar_documents(query)
    prompt = build_prompt(query, retrieved_docs)

    response = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=[
            {"role": "system", "content": "당신은 민원 정보를 잘 알려주는 공공서비스 상담 챗봇입니다."},
            {"role": "user", "content": prompt}
        ],
        temperature=1
    )

    return response.choices[0].message.content

In [11]:
# 예시 사용
query = "몇 년 전에 친양자로 입양한 자녀가 있는데 가족 간 갈등으로 인해 입양을 취소하고 싶습니다. 어떤 절차를 따라야 하나요?"
response = answer_query(query)
print(response)

친양자 입양을 취소하고 싶으시다면, 다음 절차를 따라야 합니다.

1. **신청 자격**: 누구나 신청 가능하므로 신청하실 수 있습니다.

2. **신청 방법**: 신청은 방문 또는 우편으로 가능합니다.

3. **구비 서류**:
   - 친양자입양취소당사자의 가족관계등록부의 기본증명서와 친양자입양관계증명서 각 1통 (전산정보 처리로 확인할 수 없는 경우에 한함)
   - 친양자 입양취소재판의 등본 및 확정증명 각 1부
   - 신고인의 신분증명서 사본 (우편신고의 경우)
   - 신고인이 출석한 경우의 신고인 신분증명서, 제출인이 출석한 경우 제출인의 신분증명서

4. **신청서**: ‘친양자 입양취소신고서’를 작성하셔야 합니다. 

5. **처리 기간**: 민원 접수는 시, 구, 읍, 면에서 즉시 처리됩니다.

6. **수수료**: 이 민원은 수수료가 없습니다.

7. **법적 근거**: 이 과정은 '가족관계의 등록 등에 관한 법률(제69조, 제70조)'에 규정되어 있습니다.

이러한 절차를 통해 친양자 입양 취소를 진행하시기 바랍니다. 추가 문의가 필요하시다면 해당 가족관계등록 관서에 문의하실 수 있습니다.


In [12]:
def generate_user_questions_prompt(task_name: str, task_summary: str) -> list:
    return [
        {
            "role": "system",
            "content": (
                "당신은 실제로 어떤 민원을 넣으려는 일반 시민입니다. 아래의 민원 제목과 개요를 참고해서, "
                "현실적인 배경이 있는 질문을 5개 작성하세요. 각 질문은 실제 민원인의 처지에서 상황을 간단히 설명하고, "
                "그 상황에 맞는 질문을 하는 식이어야 합니다.\n\n"
                "예시:\n"
                "- 몇 년 전에 친양자로 입양한 자녀가 있는데 가족 간 갈등으로 인해 입양을 취소하고 싶습니다. 어떤 절차를 따라야 하나요?\n"
                "- 예전에 압류된 차량을 다시 등록하려고 하는데 구비서류가 복잡한지 궁금합니다.\n"
                "- 부모님이 돌아가시면서 상속 때문에 토지이용계획확인서를 떼야 하는데 어디서 신청하나요?\n\n"
                "이런 형식으로 사용자 상황이 담긴 질문 5개를 만들어 주세요."
            )
        },
        {
            "role": "user",
            "content": f"[사무명]: {task_name}\n[사무개요]: {task_summary}"
        }
    ]


def get_generated_questions(task_name, task_summary):
    messages = generate_user_questions_prompt(task_name, task_summary)
    response = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=messages,
        temperature=0.7
    )
    raw_output = response.choices[0].message.content
    return [q.strip("-• ").strip() for q in raw_output.strip().split("\n") if q.strip()]

In [34]:
def build_prompt_row_context(question: str, row) -> str:
    context = (
        f"- 사무명: {row['사무명']}\n"
        f"- 사무개요: {row['사무개요']}\n"
        f"- 신청유형: {row['신청유형']}\n"
        f"- 신청자 자격: {row.get('신청자 자격', '')}\n"
        f"- 수수료: {row.get('수수료', '')}\n"
        f"- 수수료내역: {row.get('수수료내역', '')}\n"
        f"- 소관부처: {row.get('소관부처', '')}\n"
        f"- 담당부서: {row.get('담당부서', '')}\n"
        f"- 접수방법: {row.get('접수방법', '')}\n"
        f"- 접수/처리: {row.get('접수/처리', '')}\n"
        f"- 근거법령: {row.get('근거법령', '')}\n"
        f"- 신청서: {row.get('신청서', '')}\n"
        f"- 구비서류: {row.get('구비서류', '')}"
    )
    return (
        f"{context}\n\n"
        f"다음 질문에 대해, 위 행(row)의 정보만을 사용하여 정확하게 답변해주세요:\n"
        f"{question}\n\n"
        f"[지시사항]\n"
        f"❗ 절대 하지 마세요:\n"
        f"- '문서에 명시되어 있지 않다', '문서에 포함되어 있지 않다', '해당 문서에서 확인할 수 없다', '서식에서 확인하라' 등 문서의 내용을 참조하거나 문서에 없다고 말하는 표현은 **어떤 형태로든 절대 사용하지 마세요.**\n"
        f"- 그 대신, 정보가 확인되지 않거나 불명확한 경우에는, 반드시 **해당 행의 '소관부처'와 '담당부서'의 실제 명칭을 사용하여 문의를 유도하는 표현**으로 바꿔주세요. 예: '이와 관련된 규정은 명시되어 있지 않지만, 보다 정확한 안내를 위해 농림축산식품부의 축산환경자원과에 문의하시기 바랍니다.'\n\n"
        f"✅ 반드시 지켜주세요:\n"
        f"- 위 행(row)의 정보만을 바탕으로 작성하세요.\n"
        f"- 외부 지식, 일반 상식, 추론을 포함하지 마세요.\n"
        f"- 문서에 있는 내용을 '없다'고 하지 마세요.\n"
        f"- 민원인의 질문에 친절하고 이해하기 쉬운 말투로 설명해주세요.\n"
        f"- 아래 항목들을 가능한 다음과 같은 순서로, 자연스럽게 녹여 설명해주세요:\n"
        f"  1. 사무명, 소관 부처 및 담당 부서\n"
        f"  2. 수수료 및 수수료 내역\n"
        f"  3. 신청 자격 및 접수 방법\n"
        f"  4. 필요 서류\n"
        f"  5. 신청서 양식\n"
        f"- 마크다운 기호(**, -, # 등)는 사용하지 마세요.\n\n"
        f"[답변]:"
    )

def answer_query_with_row(question: str, row) -> str:
    prompt = build_prompt_row_context(question, row)

    response = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=[
            {"role": "system", "content": "당신은 민원 정보를 잘 알려주는 공공서비스 상담 챗봇입니다."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )
    return response.choices[0].message.content


In [ ]:
import pandas as pd

qa_pairs = []

original_df = pd.read_excel("행정안전부_민원안내정보_20250327.xlsx")

for i in range(1400, 1600):
    meta = metadatas[i]
    doc = documents[i]

    # 사무명과 문서 유효성 확인
    if isinstance(meta.get("사무명"), str) and isinstance(doc, str):
        task_name = meta["사무명"]

        # 문서에서 사무개요 줄 추출
        task_summary_line = [line for line in doc.split("\n") if "[사무개요]" in line]
        if not task_summary_line:
            continue
        task_summary = task_summary_line[0].replace("[사무개요]", "").strip()

        # ✅ 질문 생성 (GPT-4o Mini)
        questions = get_generated_questions(task_name, task_summary)

        # ✅ 행 전체 context를 구성 (해당 행의 모든 정보가 담긴 텍스트 사용)
        row_series = original_df.iloc[i]

        # ✅ 질문별 답변 생성
        for q in questions:
            print(f"\n🟨 질문: {q}")
            answer = answer_query_with_row(q, row_series)
            print(f"🟩 답변: {answer}")

            qa_pairs.append({
                "사무명": task_name,
                "질문": q,
                "답변": answer
            })

# 결과를 DataFrame으로 변환
new_df = pd.DataFrame(qa_pairs)



🟨 질문: 1. 최근에 건설기계사업을 운영하면서 사업장 주소가 변경되었습니다. 변경된 주소로 사업자 신고를 하려는데, 어떤 서류를 준비해야 할까요?
🟩 답변: 건설기계사업자 변경신고에 대해 안내드리겠습니다. 이 민원은 건설기계사업을 신고한 자가 신고한 사항에 변경이 있을 경우, 30일 이내에 신고해야 하는 절차입니다. 이 사무는 국토교통부의 건설산업과에서 담당하고 있습니다.

신고를 위해서는 1건당 5,000원의 수수료가 필요합니다. 누구나 신청이 가능하며, 접수 방법은 방문, 우편, 인터넷으로 진행할 수 있습니다.

변경된 주소로 사업자 신고를 하시려면 다음과 같은 서류를 준비하셔야 합니다. 첫째, 변경사실을 증명하는 서류가 필요합니다. 둘째, 사업자등록증도 제출하셔야 하는데, 이는 등록증 기재사항에 변경이 있는 경우에 해당합니다. 또한 공무원 확인사항으로 주민등록정보와 법인 등기사항 증명서(법인의 경우)도 확인됩니다.

신청서 양식은 '건설기계사업자 변경 신고서'로, 건설기계관리법 시행규칙의 별지서식 33호를 참고하셔야 합니다.

보다 정확한 안내를 원하신다면, 국토교통부의 건설산업과에 문의하시기 바랍니다.

🟨 질문: 2. 제 사업에 사용하던 건설기계가 고장나서 폐기 처분을 하게 되었는데, 이를 신고해야 하는지 궁금합니다. 폐기 처리 후에는 어떤 절차를 따라야 하는지 알려주세요.
🟩 답변: 건설기계사업자 변경신고와 관련하여 안내드립니다. 이 신고는 건설기계사업을 신고한 자가 신고한 사항에 변경이 있는 경우 30일 이내에 신고해야 하는 민원사무입니다. 해당 민원은 국토교통부의 건설산업과에서 담당하고 있습니다.

신고 시 수수료는 5,000원이며, 누구나 신청 가능합니다. 접수 방법은 방문, 우편, 인터넷으로 가능하니 편리한 방법을 선택하시면 됩니다.

신고를 위해 필요한 서류는 다음과 같습니다. 첫째, 변경사실을 증명하는 서류가 필요합니다. 둘째, 사업자등록증이 필요하며, 이는 등록증 기재사항에 변경이 있는 경우에 한합니다. 공무원이 확인할 사항으로는 주

KeyboardInterrupt: 

In [47]:
new_df.to_excel("1300_1399질의응답데이터셋.xlsx", index = False)

In [47]:
import streamlit as st
# Streamlit UI
st.title("📄 민원 상담 GPT 챗봇")
query = st.text_input("궁금한 민원 내용을 입력해주세요:")

if st.button("답변 받기") and query:
    with st.spinner("GPT가 답변을 생성 중입니다..."):
        answer = generate_answer(query)
        st.success(answer)

2025-04-30 23:25:01.712 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-30 23:25:01.752 
  command:

    streamlit run /Users/type/textmining-pj/.venv/lib/python3.13/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-04-30 23:25:01.752 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-30 23:25:01.752 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-30 23:25:01.753 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-30 23:25:01.753 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-30 23:25:01.753 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-30 23:25:01.753 Sessi

NameError: name 'index' is not defined